In [1]:
import io
import os
import json
import distutils.dir_util
from collections import Counter

import numpy as np

def write_json(data, fname):
    def _conv(o):
        if isinstance(o, np.int64) or isinstance(o, np.int32):
            return int(o)
        raise TypeError

    parent = os.path.dirname(fname)
    distutils.dir_util.mkpath("./" + parent)
    with io.open("./" + fname, "w", encoding="utf8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)


def load_json(fname):
    with open(fname, encoding='utf8') as f:
        json_obj = json.load(f)

    return json_obj


def debug_json(r):
    print(json.dumps(r, ensure_ascii=False, indent=4))

In [17]:
import pickle
import numpy as np
import pandas as pd
import scipy.sparse as spr
from collections import Counter
from scipy.sparse import hstack, vstack
from tqdm.auto import tqdm

def remove_seen(seen, l):
    seen = set(seen)
    return [x for x in l if not (x in seen)]


def most_popular(playlists, col, topk_count):
    c = Counter()

    for doc in playlists:
        c.update(doc[col])

    topk = c.most_common(topk_count)
    return c, [k for k, v in topk]

song_meta = pd.read_json("./data/song_meta.json")
train = pd.read_json("./data/train.json")
test = pd.read_json("./data/test.json")

In [8]:
train['istrain'] = 1
test['istrain'] = 0

n_train = len(train)
n_test = len(test)

# train + test
plylst = pd.concat([train, test], ignore_index=True)

# playlist id
plylst["nid"] = range(n_train + n_test)

# id <-> nid
plylst_id_nid = dict(zip(plylst["id"],plylst["nid"]))
plylst_nid_id = dict(zip(plylst["nid"],plylst["id"]))

In [9]:
plylst_tag = plylst['tags']
tag_counter = Counter([tg for tgs in plylst_tag for tg in tgs])
tag_dict = {x: tag_counter[x] for x in tag_counter}

tag_id_tid = dict()
tag_tid_id = dict()
for i, t in enumerate(tag_dict):
    tag_id_tid[t] = i
    tag_tid_id[i] = t

n_tags = len(tag_dict)

plylst_song = plylst['songs']
song_counter = Counter([sg for sgs in plylst_song for sg in sgs])
song_dict = {x: song_counter[x] for x in song_counter}

song_id_sid = dict()
song_sid_id = dict()
for i, t in enumerate(song_dict):
    song_id_sid[t] = i
    song_sid_id[i] = t

n_songs = len(song_dict)

In [10]:
plylst['songs_id'] = plylst['songs'].map(lambda x: [song_id_sid.get(s) for s in x if song_id_sid.get(s) != None])
plylst['tags_id'] = plylst['tags'].map(lambda x: [tag_id_tid.get(t) for t in x if tag_id_tid.get(t) != None])

plylst_use = plylst[['istrain','nid','updt_date','songs_id','tags_id']]

# 곡의 개수와 태그의 개수를 할당
plylst_use.loc[:,'num_songs'] = plylst_use['songs_id'].map(len)
plylst_use.loc[:,'num_tags'] = plylst_use['tags_id'].map(len)
plylst_use = plylst_use.set_index('nid')

/Users/jonghyunlee/anaconda3/envs/coldmelon/lib/python3.6/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/jonghyunlee/anaconda3/envs/coldmelon/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [11]:
# train test split
plylst_train = plylst_use.iloc[:n_train,:]
plylst_test = plylst_use.iloc[n_train:,:]

In [12]:
# csr (compressed sparse row matrix 생성)
row = np.repeat(range(n_train), plylst_train['num_songs'])
col = [song for songs in plylst_train['songs_id'] for song in songs]
dat = np.repeat(1, plylst_train['num_songs'].sum())
train_songs_A = spr.csr_matrix((dat, (row, col)), shape=(n_train, n_songs))

row = np.repeat(range(n_train), plylst_train['num_tags'])
col = [tag for tags in plylst_train['tags_id'] for tag in tags]
dat = np.repeat(1, plylst_train['num_tags'].sum())
train_tags_A = spr.csr_matrix((dat, (row, col)), shape=(n_train, n_tags))

# csr (compressed sparse row matrix 생성)
row = np.repeat(range(n_test), plylst_test['num_songs'])
col = [song for songs in plylst_test['songs_id'] for song in songs]
dat = np.repeat(1, plylst_test['num_songs'].sum())
test_songs_A = spr.csr_matrix((dat, (row, col)), shape=(n_test, n_songs))

row = np.repeat(range(n_test), plylst_test['num_tags'])
col = [tag for tags in plylst_test['tags_id'] for tag in tags]
dat = np.repeat(1, plylst_test['num_tags'].sum())
test_tags_A = spr.csr_matrix((dat, (row, col)), shape=(n_test, n_tags))

In [13]:
train_merged_feature_csr = hstack([train_songs_A, train_tags_A])
test_merged_feature_csr  = hstack([test_songs_A, test_tags_A])

train_songs_A_T = train_songs_A.T.tocsr()
train_tags_A_T  = train_tags_A.T.tocsr()

whole_datasets_csr       = vstack([train_merged_feature_csr, test_merged_feature_csr])
whole_datasets_csr_T     = whole_datasets_csr.T.tocsr()

In [14]:
whole_datasets_csr_T.shape

(656082, 125811)

In [15]:
def rec(pids):
    res = []

    for pid in tqdm(pids):
        p = np.zeros((n_songs,1))
        p[plylst_test.loc[pid,'songs_id']] = 1

        val = train_songs_A.dot(p).reshape(-1)

        songs_already = plylst_test.loc[pid, "songs_id"]
        tags_already = plylst_test.loc[pid, "tags_id"]

        cand_song = train_songs_A_T.dot(val)
        cand_song_idx = cand_song.reshape(-1).argsort()[-250:][::-1]

        cand_song_idx = cand_song_idx[np.isin(cand_song_idx, songs_already) == False][:100]
        rec_song_idx = [song_sid_id[i] for i in cand_song_idx]

        cand_tag = train_tags_A_T.dot(val)
        cand_tag_idx = cand_tag.reshape(-1).argsort()[-15:][::-1]

        cand_tag_idx = cand_tag_idx[np.isin(cand_tag_idx, tags_already) == False][:10]
        rec_tag_idx = [tag_tid_id[i] for i in cand_tag_idx]

        res.append({
            "id": plylst_nid_id[pid],
            "songs": rec_song_idx,
            "tags": rec_tag_idx
        })
        
    return res

res = rec(plylst_test.index)

In [16]:
os.mkdir("final_results")
write_json(res, "./final_results/results_MF.json")

In [27]:
from gensim.models import Word2Vec
from gensim.models.keyedvectors import WordEmbeddingsKeyedVectors

class PlaylistEmbedding:
    def __init__(self, FILE_PATH):
        self.FILE_PATH = FILE_PATH
        self.min_count = 3
        self.size = 300
        self.window = 300
        self.sg = 5
        self.p2v_model = WordEmbeddingsKeyedVectors(self.size)
        with open(os.path.join(FILE_PATH, 'train.json'), encoding="utf-8") as f:
            self.train = json.load(f)
        with open(os.path.join(FILE_PATH, 'test.json'), encoding="utf-8") as f:
            self.val = json.load(f)
        with open(os.path.join("./final_results/", 'results_MF.json'), encoding="utf-8") as f:
            self.most_results = json.load(f)
            
    def get_dic(self, train, val):
        song_dic = {}
        tag_dic = {}
        data = train + val
        for q in tqdm(data):
            song_dic[str(q['id'])] = q['songs']
            tag_dic[str(q['id'])] = q['tags']
        self.song_dic = song_dic
        self.tag_dic = tag_dic
        total = list(map(lambda x: list(map(str, x['songs'])) + list(x['tags']), data))
        total = [x for x in total if len(x)>1]
        self.total = total
        
    def get_w2v(self, total, min_count, size, window, sg):
        w2v_model = Word2Vec(total, min_count = min_count, size = size, window = window, sg = sg)
        self.w2v_model = w2v_model
            
    def update_p2v(self, train, val, w2v_model):
        ID = []   
        vec = []
        for q in tqdm(train + val):
            tmp_vec = 0
            if len(q['songs'])>=1:
                for song in q['songs'] + q['tags']:
                    try: 
                        tmp_vec += w2v_model.wv.get_vector(str(song))
                    except KeyError:
                        pass
            if type(tmp_vec)!=int:
                ID.append(str(q['id']))    
                vec.append(tmp_vec)
        self.p2v_model.add(ID, vec)
    
    def get_result(self, p2v_model, song_dic, tag_dic, most_results, val):
        answers = []
        for n, q in tqdm(enumerate(val), total = len(val)):
            try:
                most_id = [x[0] for x in p2v_model.most_similar(str(q['id']), topn=200)]
                get_song = []
                get_tag = []
                for ID in most_id:
                    get_song += song_dic[ID]
                    get_tag += tag_dic[ID]
                get_song = list(pd.value_counts(get_song)[:200].index)
                get_tag = list(pd.value_counts(get_tag)[:20].index)
                answers.append({
                    "id": q["id"],
                    "songs": remove_seen(q["songs"], get_song)[:100],
#                     "songs": most_results[n]['songs'][:100],
                    "tags": remove_seen(q["tags"], get_tag)[:10],
                })
            except:
                answers.append({
                  "id": most_results[n]["id"],
                  "songs": most_results[n]['songs'],
                  "tags": most_results[n]["tags"],
                }) 
        # check and update answer
        for n, q in enumerate(answers):
            if len(q['songs'])!=100:
                answers[n]['songs'] += remove_seen(q['songs'], self.most_results[n]['songs'])[:100-len(q['songs'])]
            if len(q['tags'])!=10:
                answers[n]['tags'] += remove_seen(q['tags'], self.most_results[n]['tags'])[:10-len(q['tags'])]  
        self.answers = answers
    
    def run(self):
        self.get_dic(self.train, self.val)
        self.get_w2v(self.total, self.min_count, self.size, self.window, self.sg)
        self.update_p2v(self.train, self.val, self.w2v_model)
        self.get_result(self.p2v_model, self.song_dic, self.tag_dic, self.most_results, self.val)
        write_json(self.answers, 'results.json')

In [28]:
FILE_PATH = './data/'
item2vec = PlaylistEmbedding(FILE_PATH)
item2vec.run()